<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/ML2024_ex02notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex02notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024)


----
## カリフォルニアの住宅価格の予測
----

回帰のための教師あり学習の応用として，平面当てはめによってアメリカカリフォルニアの住宅価格を予測してみましょう．

---
### 準備

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

ここで使うのは，California housing dataset という，アメリカカリフォルニアの住宅価格を予測する回帰問題のデータセットです．

参考
- https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset
- https://atmarkit.itmedia.co.jp/ait/articles/2201/31/news042.html

#### データの入手

次のセルを実行すると，データセットを入手できます．

参考: scikit-learn の [sklearn.datasets.fetch_california_housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) 関数

In [ ]:
# as_frame を True にすると pandas.DataFrame 形式で取得できる
data = fetch_california_housing(as_frame=True)
dfX = data['data']
dfX

見て分かるように，8つの値から成るデータが20640件あります．これらは回帰モデルへの入力となります．
1件のデータは，一定の地域内に存在する住宅および住人について求めた次の8つの値から成ります（詳細は↑の参考リンク先参照）．

1. MedInc: 所得の中央値
1. HouseAge: 築年数の中央値
1. AveRooms: 部屋数の平均値
1. AveBedrms: 寝室数の平均値
1. Population: 人口
1. AveOccup: 世帯数の平均値
1. Latitude: 緯度
1. Longitude: 経度

予測対象はその地域の住宅価格の中央値で，次のような値です．

In [ ]:
dfY = data['target']
dfY

次のセルを実行すると，このデータセットの中で住宅価格が最小と最大のデータを表示させることができます．

In [ ]:
# Y の値が最小と最大の行を表示
imin, imax = dfY.argmin(), dfY.argmax()
dfX.iloc[[imin, imax], :]

1行目が住宅価格最小の地域，2行目が最大の地域のデータです．

#### 前処理

以下のセルを実行すると，データセットを，パラメータの学習に使うデータ（学習データ）と予測精度の検証に使うデータ（検証データ）に分割します．

参考: [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)




In [ ]:
# 全データの 8 割を学習データとし，残りを検証データとする
#   random_state を指定しなければ実行のたびに分割の仕方が変わるので毎回異なる実験結果となるが，
#   ここでは一つの値に固定しているで，何度やっても（当然，誰がやっても）同じ条件で実験できる
XL, XV, yL, yV = train_test_split(dfX.to_numpy(), dfY.to_numpy(), train_size=0.8, random_state=0)
print(XL.shape, yL.shape, XV.shape, yV.shape)

入力となる8つの変数のばらつき方が変数ごとに違いすぎるので，各変数の平均が 0 で分散が 1 となるようにする前処理（標準化）を適用しておきます．

参考: [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
# XLの変数ごとの平均と分散を求める
scaler = StandardScaler()
scaler.fit(XL)
print(scaler.mean_) # 平均
print(scaler.var_) # 分散

# 求めた平均と分散を使って XL, XV を標準化
XL2 = scaler.transform(XL)
XV2 = scaler.transform(XV)

---
### 最小二乗法による平面当てはめ

In [ ]:
# XL2 に 1 を付け足して Nx(D+1) 行列 X をつくる
X = np.vstack((np.ones(len(XL2)), XL2.T)).T
print(X.shape)
print(X)

In [ ]:
# 正規方程式を解く
XTX = X.T @ X  # 正規方程式の左辺の(D+1)x(D+1)の行列
XTY = X.T @ yL  # 正規方程式の右辺の(D+1)x1の行列
w = np.linalg.solve(XTX, XTY) # 連立方程式を解く
print(f'{len(w)}個のパラメータの値は')
print(w)

学習データに対して二乗誤差の和を最小にするパラメータ $\mathbf{w} = (w_0, w_1, \ldots, w_{8})$ の値が得られました．

MedInc （を標準化した値）に対する係数が 0.83 と大きいので，所得が高い地域ほど住宅価格も高くなる傾向にあるようです．一方，Latitude と Longitude の係数は負です．Latitude は北に行くほど大きくなりますので、南の方が住宅価格が高くなりがちなようです．

---
### 予測してみる


得られたパラメータでどの程度うまく住宅価格を予測できるか，学習データと検証データのそれぞれで確認してみましょう．

In [ ]:
# 学習データに対する予測値を求める
XXL = np.vstack((np.ones(len(XL2)), XL2.T)).T
YYL = XXL @ w
print(XXL.shape, YYL.shape)

# 最初の10件を表示
for i in range(10):
    print(f'{i}  入力', end=' ')
    for x in XL2[i, :]:
        print(f'{x: .3f}', end=' ')
    print(f'  予測値 {YYL[i]:.3f}  正解 {yL[i]:.3f}  誤差の二乗 {(YYL[i] - yL[i])**2:.3f}')

In [ ]:
# 検証データに対する予測値を求める
XXV = np.vstack((np.ones(len(XV2)), XV2.T)).T
YYV = XXV @ w
print(XXV.shape, YYV.shape)

# 最初の10件を表示
for i in range(10):
    print(f'{i}  入力', end=' ')
    for x in XV2[i, :]:
        print(f'{x: .3f}', end=' ')
    print(f'  予測値 {YYV[i]:.3f}  正解 {yV[i]:.3f}  誤差の二乗 {(YYV[i] - yV[i])**2:.3f}')

それなりに予測できているものもあれば，大きく外しているものもあるようです．

どれくらいうまく予測できているか定量的に判断するために，学習データと検証データのそれぞれについて，予測値と正解の値との間の平均二乗誤差（残差平方和をデータ数で割った値）を求めると，次のようになります．

In [ ]:
# 学習データ/検証データに対する出力と正解との間の平均二乗誤差
msqeL = np.mean((YYL - yL)**2)
msqeV = np.mean((YYV - yV)**2)
print(f'(学習データの平均二乗誤差) = {msqeL:.4f}  (検証データの平均二乗誤差) =  {msqeV:.4f}')

学習に使わなかった検証データに対する平均二乗誤差の方が，少しだけ大きいようです．

#### ★ やってみよう

結果等はノート等（紙媒体）にメモしておこう

1. ここで考えているモデルにはパラメータはいくつあるか
1. 検証データの最初の10件のうち，誤差の二乗を基準として，予測が最もうまくいっているものと最もうまくいっていないものはどれか（値をメモしておこう）．


---
## 顔画像からの年齡推定
---



---
### はじめに

平面当てはめの応用例として，顔画像からそこに写った人物の年齡を推定するデモを作ってみました．
画素値を並べたものに平面を当てはめる，という単純なモデルですので，推定精度は高くありません．
推定精度を高めたければ，もっと複雑なモデルを使うことになります．

人の顔が中央に大きく写った幅64画素高さ64画素の3チャンネルカラー画像 38,138 枚があります．
これらの画像に写った人物の年齡を表す整数値も与えられています．
このとき，一つの画像の画素値を1列にならべて $64\times 64\times 3 = 12288$ 次元ベクトルとみなせば，そこから年齡を予測する問題を重回帰分析の問題として定式化できます．

ひとつの顔に対応する $12288$ 次元ベクトルに $1$ を付け足した $12289$ 次元ベクトルを $\mathbf{x}$ とおき，そのひとの年齡を $y$ とおくとき，

$$
y \approx \mathbf{w}\cdot\mathbf{x}
$$

となるような $(D+1)$次元ベクトル $\mathbf{w}$ を最小二乗法によって求める，というわけです（実際には，データの次元数が大きすぎるので，主成分分析を利用した次元削減も併用しています．詳しくはこの notebook の後の方に書いてます）．

この notebook では，上記の設定で既に学習済みの $\mathbf{w}$ を用いて，適当な画像 $\mathbf{x}$ を与えて年齡推定をさせてみます．


---
### 準備

In [ ]:
# 必要なパッケージのインポート
import numpy as np
import cv2  # Python による「コンピュータビジョン(Computer Vision)」のためのライブラリ OpenCV のパッケージをインポート

# 顔検出器の準備
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

次の3つのセルではあとで使う関数を定義しています．これらのセルを実行しただけでは何も起こりませんが，あらかじめ実行して関数を定義しておかないと後のセルが動きません．

In [ ]:
# Colab へのファイルアップロードを実行する関数
#
def uploadToColab():
    try:
        from google.colab import files
        rv = files.upload()
    except:
        print('このコードは Colab 以外の環境では実行できないよ．')


In [ ]:
# OpenCVの形式の画像を表示する関数
#
def imshow(img):
    try:
        from google.colab.patches import cv2_imshow
        cv2_imshow(img) # Colab上で実行している場合
    except:
        cv2.imshow(img)  # それ以外の場合

In [ ]:
# 与えられた画像の中から顔を検出する
#
def faceDetector(img, faceCascade, maxSize=400):

    h, w = img.shape[:2]

    # (短辺の長さ) <= maxSize にリサイズ
    if min(w, h) > maxSize:
        if w <= h:
            w2, h2 = maxSize, h*maxSize//w
        else:
            w2, h2 = w*maxSize//h, maxSize
        imgDisp = cv2.resize(img, (w2, h2))
    else:
        imgDisp = np.copy(img)

    # 顔検出を実行
    imgGray = cv2.cvtColor(imgDisp, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(imgGray, 1.1, 4)
    nFace = len(faces)

    # 検出した顔のうち最初のひとりを選んで切り取り
    if nFace > 0:
        print(faces)
        x, y, ww, hh = faces[0]
        imgFace = np.copy(imgDisp[y:y+hh, x:x+ww, :])
        cv2.rectangle(imgDisp, (x, y), (x+ww, y+hh), color=(0, 255, 0), thickness=2)
        return nFace, imgDisp, imgFace
    else:
        return nFace, imgDisp, None


$\mathbf{w}$ などのパラメータを読み込みます．

In [ ]:
# takataka のウェブサイトからパラメータを格納したファイルを Colab へダウンロード
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/ageestimation.npz

import os
path = 'ageestimation.npz'
if os.path.exists(path):
    # パラメータを読み込む
    params = np.load(path)
    print(f'ファイル {path} を読み込みました')
    Xm  = params['Xm']  # 平均ベクトル
    eve = params['eve'] # 主成分分析で得られた固有ベクトル
    w   = params['w']   # 重回帰分析で得られたパラメータ
    print(Xm.shape, eve.shape, w.shape)
else:
    print(f'ファイル {path} の読み込みに失敗したようです．再実行してみてください')

---
### 実験

(1) 画像を Colab へアップロードします．次のような画像にしてください

- ひとりのひとのほぼ正面を向いた顔全体が写ってる．複数人を検出した場合，ひとりだけ抜き出します（あとで条件に合わない画像をわざと与えて実験してみるのもよいでしょう）．
- 画像全体に顔がドアップで写っているような場合はうまく顔検出ができないかもしれません．顔がもう少し小さく写ってる画像を探してみましょう．
- ファイル名に空白やマルチバイト文字（日本語など）が含まれているとうまく動作しない場合があるので，自分のPCの方で適当な名前に変えておく（拡張子は変えてはいけない）．


In [ ]:
# 以下の 0 を 1 に修正してセルを実行しよう
#
if 1 == 0:
    # Colab へファイルをアップロード
    uploadToColab()
# ls コマンドでファイルを一覧
! ls

上のセルを実行してファイルをアップロードできたら，次のセルのファイル名の部分にその名前を指定して，読み込ませましょう．

In [ ]:
# 画像を読み込む．`hoge.jpg` を自分がアップロードしたファイルの名前に修正
img = cv2.imread('hoge.jpg')

(2) 顔検出を実行して顔領域を切り取った画像を作る

In [ ]:
numFace, imgDisp, imgFace = faceDetector(img, faceCascade, maxSize=400)
if numFace > 0:
    imshow(imgDisp)
    imshow(imgFace)
else:
    imshow(imgDisp)
    print('顔を検出できませんでした')

(3) 顔を所定の大きさにリサイズしてから次元削減し，年齡推定を実行．

In [ ]:
# 顔画像を 64 x 64 にリサイズして 12288次元ベクトルに
xvec = cv2.resize(imgFace, (64, 64)).reshape(-1)
print(xvec.shape)

# PCAを利用した次元削減
yvec = (xvec - Xm) @ eve
print(yvec.shape)

# 予測値の計算
yvec1 = np.concatenate(([1], yvec)) # 先頭に 1 をくっつける
z = w @ yvec1

# 結果の表示
print(f'このひとの年齡の推定値は {z:.1f} です')

### 解説

この年齡推定の仕組みは，次のようにして作りました．以下のうち，(3), (4), (5) ではデータの次元を削減するために主成分分析を実行しています．次元削減や主成分分析については，「多変量解析及び演習」で登場していますので，興味のある方は [2023年度「多変量解析及び演習」](https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2023) へどうぞ．「機械学習II」でも登場する予定です．

(1) [IMDB-WIKI dataset](https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/) という研究用画像データセットの中に，Wikipedia から集めた顔画像とその性別，生年月日，撮影年などが含まれたデータ（画像 62,328枚）があったので，これを利用．

(2) 生年月日と撮影年から年齡を算出．顔がちゃんと写ってなかったり年齡の推定値がおかしかったりするデータを削除．顔の領域のみを抜き出す．得られた画像は 38138 枚．この前処理の部分では，こちらのサイトを参考にさせてもらいました：
[顔画像から年齢・性別を推定するためのデータセットIMDB-WIKI](https://qiita.com/yu4u/items/a2410f46669c5f20ee8e)

(3) 得られた画像を $64\times 64$ に縮小してデータ行列 `X` を作成．
`X.shape` は `(38138, 12288)`．

(4) `X` の平均を求めてこれを `X` から引き，特異値分解経由で主成分分析のための固有値・固有ベクトルを算出．次のスペックの Mac で `np.linalg.svd` の実行に23分．
- CPU: Apple M1 Pro
- メモリ: 32GB

(5) 累積寄与率を調べると，2151次元ではじめて99%を超えていたので，2151次元に次元削減した．

(6) そのデータを用いて最小二乗法の解を計算．上記の Mac で `np.linalg.lsesq` の実行に 28 秒．

(7) `X` の平均（12288次元），固有ベクトル（12288x2151），重回帰のパラメータ（2152次元）をファイルに保存．

また，前処理として画像の中から顔を検出する処理も行っています．こちらは，「コンピュータビジョン(Computer Vision)」のためのライブラリ OpenCV の中にある顔検出ライブラリを使っています．
これは，画像を顔とそれ以外のものの2つに分類する仕組みで，やはり機械学習の方法で作られています．